In [2]:
# pip install --upgrade spark_ai_python

In [3]:
from sparkai.llm.llm import ChatSparkLLM, ChunkPrintHandler
from sparkai.core.messages import ChatMessage

In [4]:
# pip install dotenv

In [5]:
import  os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [6]:
# print(os.environ["SPARK_API_SECRET"])

In [7]:
# !pip install django.utils.formats

In [8]:
from datetime import datetime
from time import mktime
from wsgiref.handlers import format_date_time
import zoneinfo
import hmac
import hashlib
import base64
from urllib.parse import urlencode
from urllib.parse import urlparse

# 获取当前时间
cur_time = datetime.now()

# 设置时区为上海（CST，UTC+8）
shanghai_tz = zoneinfo.ZoneInfo("Asia/Shanghai")

# 将当前时间转换为上海时区
localized_time = cur_time.replace(tzinfo=shanghai_tz)

# 转换为时间戳
timestamp = mktime(localized_time.timetuple())

# 格式化时间戳
date = format_date_time(timestamp)

tmp = "host: " + "spark-api.xf-yun.com" + "\n"
tmp += "date: " + date + "\n"
tmp += "GET " + "/v4.0/chat" + " HTTP/1.1"
tmp_sha = hmac.new(os.environ["SPARK_API_SECRET"].encode('utf-8'), tmp.encode('utf-8'), digestmod=hashlib.sha256).digest()
signature = base64.b64encode(tmp_sha).decode(encoding='utf-8')
authorization_origin = f"api_key='{os.environ["SPARK_API_KEY"]}', algorithm='hmac-sha256', headers='host date request-line', signature='{signature}'"
authorization = base64.b64encode(authorization_origin.encode('utf-8')).decode(encoding='utf-8')
v = {
		"authorization": authorization, # 上方鉴权生成的authorization
        "date": date,  # 步骤1生成的date
    	"host": "wss://spark-api.xf-yun.com" # 请求的主机名，根据具体接口替换
}
url = "wss://spark-api.xf-yun.com/{}/chat?" + urlencode(v)

In [9]:
# print(urlparse('wss://spark-api.xf-yun.com/v4.0/chat').path)

In [10]:
# 定义spark Spark4.0 Ultra SDK
def gen_spark_params(model):
    '''
    构造星火大模型参数
    '''
    model_params_dict = {
        # v1.5 版本
        "v1.5": {
            "domain": "general", # 用于配置大模型版本
            "spark_url": url.format("v1.1") # 云端环境的服务地址
        },
        # v2.0 版本
        "v2.0": {
            "domain": "generalv2", # 用于配置大模型版本
            "spark_url": url.format("v2.1") # 云端环境的服务地址
        },
        # v3.0 版本
        "v3.0": {
            "domain": "generalv3", # 用于配置大模型版本
            "spark_url": url.format("v3.1") # 云端环境的服务地址
        },
        # v3.5 版本
        "v3.5": {
            "domain": "generalv3.5", # 用于配置大模型版本
            "spark_url": url.format("v3.5") # 云端环境的服务地址
        },
        # v4.0 版本
        "v4.0": {
            "domain": "4.0Ultra", # 用于配置大模型版本
            "spark_url": url.format("v4.0") # 云端环境的服务地址
        }
    }
    return model_params_dict[model]

In [11]:


def get_completion(prompt, model='v4.0', temperature=0.1):
   '''
   获取星火大模型的调用结果

   请求参数：
        prompt：对应提示词
        model：调用的模型，默认为v4.0，也可以根据实际需求选择合适的模型
        temperature：模型输出的温湿度系数，
   ''' 
   spark = ChatSparkLLM(
        spark_api_url=gen_spark_params(model)["spark_url"],
        spark_app_id=os.environ["SPARK_APPID"],
        spark_api_key=os.environ["SPARK_API_KEY"],
        spark_api_secret=os.environ["SPARK_API_SECRET"],
        spark_llm_domain=gen_spark_params(model)["domain"],
        streaming=False,
    )
   messages = [ChatMessage(
        role="user",
        content=prompt
    )]
   handler = ChunkPrintHandler()
   response = spark.generate([messages], callbacks=[handler])
   return response

In [12]:
get_completion("今天天气很不错，帮我写一首诗，诗中需要包含当前季节的相关元素")

LLMResult(generations=[[ChatGeneration(text='《春光》\n柳绿花红映日辉，莺歌燕舞闹芳菲。\n清风拂面心神爽，满目春光尽意归。\n\n注释：翠绿的柳树和盛开的鲜花在阳光下交相辉映，黄莺在歌唱，燕子在飞舞，它们都在热闹地迎接这芬芳美丽的春天。轻柔的春风拂过脸庞，让人感到心情舒畅，精神振奋。放眼望去，到处都是春天的美景，让人心满意足地沉醉其中。', message=AIMessage(content='《春光》\n柳绿花红映日辉，莺歌燕舞闹芳菲。\n清风拂面心神爽，满目春光尽意归。\n\n注释：翠绿的柳树和盛开的鲜花在阳光下交相辉映，黄莺在歌唱，燕子在飞舞，它们都在热闹地迎接这芬芳美丽的春天。轻柔的春风拂过脸庞，让人感到心情舒畅，精神振奋。放眼望去，到处都是春天的美景，让人心满意足地沉醉其中。'))]], llm_output={'token_usage': {'question_tokens': 16, 'prompt_tokens': 16, 'completion_tokens': 106, 'total_tokens': 122}}, run=[RunInfo(run_id=UUID('86a651bb-94fb-40ff-ba13-ef63611c1d73'))])